In [ ]:
pip install tensorflow-datasets

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [ ]:
(train_ds, validation_ds), ds_info = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:]'],
    as_supervised= True,
    with_info= True
)

In [ ]:
# PRE-PROCESSING THE IMAGE
def preprocess(image, label):
    image = tf.image.resize(image, [150, 150]) # resizing the images
    image = tf.cast(image, tf.float32)/255.0 # normalize pixel values, everything is between zero and one
    return image, label

In [ ]:
def augument(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_brightness(image, max_delta=0.5)
    return image, label

In [ ]:
train_ds = train_ds.map(preprocess).map(augument).batch(32).prefetch(buffer_size=tf.data.AUTOTUNE) #TRAIN DATASET
validation_ds = validation_ds.map(preprocess).batch(32).prefetch(buffer_size=tf.data.AUTOTUNE) # VALIDATION DATASET

In [ ]:
# LOAD VGG16 MODEL PRE-TRAINED ON IMAGENET DATA
vgg_base = VGG16(weights='imagenet', include_top=False,input_shape=(150, 150, 3))
## FREEZING LAYERS
for layer in vgg_base.layers:
    layer.trainable = False
# FINE TUNING - CREATING A NEW MODEL ON TOP
model = Sequential([
    vgg_base,
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.15),
    Dense(1, activation='sigmoid')
])

In [ ]:
# COMPILING THE MODEL
model.compile(optimizer='adam',loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
# Train the model
callback = EarlyStopping(monitor='val_accuracy', patience=5)
history = model.fit(train_ds, epochs=100, validation_data=validation_ds, callbacks=[callback])

In [ ]:
def display_multiple_predictions(dataset, model, num_images=5):
    plt.figure(figsize=(15, 3))  # Set the figure size

    for i, (img, label) in enumerate(dataset.unbatch().take(num_images)):
        # Convert the label tensor to a numpy value
        actual_label = 'Dog' if label.numpy() == 1 else 'Cat'

        # Make a prediction
        predictions = model.predict(tf.expand_dims(img, 0))
        predicted_label = 'Dog' if predictions[0][0] > 0.5 else 'Cat'

        # Plotting
        plt.subplot(1, num_images, i+1)
        plt.imshow(img.numpy().astype("int32"))
        plt.title(f'A: {actual_label}\nP: {predicted_label}')
        plt.axis('off')

    plt.show()

In [ ]:
display_multiple_predictions(validation_ds, model)